## 1. Running Pyspark in Jupyter Notebook

In [1]:
# spark-3.2.2-bin-hadoop3.2
# install findspark using pip
!pip install findspark

In [2]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

## 2. Analysis and Regression on Boston Housing Dataset

In [ ]:
# import the boston housing dataset
dataset = spark.read.csv('BostonHousing.csv', header=True, inferSchema=True)